In [ ]:
!pip install chardet

In [ ]:
!pip install pandas scikit-learn

In [ ]:
!pip install ipywidgets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import ipywidgets as widgets
from IPython.display import display, clear_output

df = pd.read_csv("/content/SampleData1.csv")

df.columns = df.columns.str.strip().str.lower()
df['tabs'] = pd.to_numeric(df['tabs'], errors='coerce')
df = df.dropna(subset=['rate quoted', 'tabs'])
df = df[df['tabs'] > 0]
df['rate_per_tab'] = df['rate quoted'] / df['tabs']
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['rate_per_tab'])

lower_bound = df['rate_per_tab'].quantile(0.01)
upper_bound = df['rate_per_tab'].quantile(0.99)
df = df[(df['rate_per_tab'] >= lower_bound) & (df['rate_per_tab'] <= upper_bound)]

cat_cols = ['company name', 'product name', 'region', 'units', 'zone']
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le

features = ['company name', 'product name', 'qty', 'region', 'units', 'zone']
X = df[features]
y = df['rate_per_tab']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.4f} INR per tab\n")

company_list = encoders['company name'].classes_
product_list = encoders['product name'].classes_
zone_list = encoders['zone'].classes_

company_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(company_list)],
    description='Company:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

zone_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(zone_list)],
    description='Zone:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

product_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(product_list)],
    description='Product:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

output = widgets.Output()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output()

            # Get selected indexes
            company_choice = company_dropdown.value
            zone_choice = zone_dropdown.value
            product_choice = product_dropdown.value

            # Map to names
            company_name = company_list[company_choice]
            zone_name = zone_list[zone_choice]
            product_name = product_list[product_choice]

            # Encode
            comp_code = encoders['company name'].transform([company_name])[0]
            prod_code = encoders['product name'].transform([product_name])[0]
            zone_code = encoders['zone'].transform([zone_name])[0]

            # Filter historical data for company+product+zone
            hist = df[(df['company name'] == comp_code) &
                      (df['product name'] == prod_code) &
                      (df['zone'] == zone_code)]

            if hist.empty or len(hist) < 3:
                print("\nWarning: Insufficient historical data for this company+product+zone combo. Using global medians.")
                qty_val = df['qty'].median()
                region_val = int(df['region'].median())
                units_val = int(df['units'].median())
                tabs_val = int(df['tabs'].median())
            else:
                qty_val = hist['qty'].median()
                region_val = int(hist['region'].median())
                units_val = int(hist['units'].median())
                tabs_val = int(hist['tabs'].median())

            print(f"\nUsing tabs value = {tabs_val} (from history)")

            # Prepare input row
            input_row = {
                'company name': comp_code,
                'product name': prod_code,
                'qty': qty_val,
                'region': region_val,
                'units': units_val,
                'zone': zone_code
            }
            input_df = pd.DataFrame([input_row], columns=features)

            # Predict rate per tab
            pred_rate_per_tab = model.predict(input_df)[0]

            # Clip prediction to historical 5th-95th percentile if history available
            if not hist.empty:
                min_rate = hist['rate_per_tab'].quantile(0.05)
                max_rate = hist['rate_per_tab'].quantile(0.95)
                pred_rate_per_tab = np.clip(pred_rate_per_tab, min_rate, max_rate)

            pred_pack_rate = pred_rate_per_tab * tabs_val

            print(f"\nPredicted rate per tab (clipped): {pred_rate_per_tab:.4f} INR")
            print(f"Predicted pack rate (rate_per_tab * tabs): {pred_pack_rate:.2f} INR")

            if not hist.empty:
                print("\nHistorical (company+product+zone) stats:")
                print(f" Mean rate_per_tab: {hist['rate_per_tab'].mean():.4f} INR")
                print(f" Mean pack rate: {hist['rate quoted'].mean():.2f} INR")
                print(f" Median tabs: {hist['tabs'].median()}")
            else:
                print("\nNo historical data available for this company+product+zone.")

# Attach observers
company_dropdown.observe(on_change)
zone_dropdown.observe(on_change)
product_dropdown.observe(on_change)

# Display UI
display(company_dropdown, zone_dropdown, product_dropdown, output)

# Trigger initial display
on_change({'type': 'change', 'name': 'value', 'new': company_dropdown.value})


Mean Absolute Error: 1.2390 INR per tab



Dropdown(description='Company:', layout=Layout(width='40%'), options=(('Aaron Babcon', 0), ('Abate Medicaments…

Dropdown(description='Zone:', layout=Layout(width='40%'), options=(('Central', 0), ('East', 1), ('North', 2), …

Dropdown(description='Product:', layout=Layout(width='40%'), options=(('Acyclovir 200 mg', 0), ('Acyclovir 400…

Output()

In [ ]:
# =============================
# 1️⃣ Imports & Data Prep + Training
# =============================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load dataset
df = pd.read_csv("/content/SampleData1.csv")

# Clean and preprocess
df.columns = df.columns.str.strip().str.lower()
df['tabs'] = pd.to_numeric(df['tabs'], errors='coerce')
df = df.dropna(subset=['rate quoted', 'tabs'])
df = df[df['tabs'] > 0]
df['rate_per_tab'] = df['rate quoted'] / df['tabs']
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['rate_per_tab'])

# Remove outliers in rate_per_tab
lower_bound = df['rate_per_tab'].quantile(0.01)
upper_bound = df['rate_per_tab'].quantile(0.99)
df = df[(df['rate_per_tab'] >= lower_bound) & (df['rate_per_tab'] <= upper_bound)]

# Encode categorical features
cat_cols = ['company name', 'product name', 'region', 'units', 'zone']
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le

# Define features & target (exclude tabs)
features = ['company name', 'product name', 'qty', 'region', 'units', 'zone']
X = df[features]
y = df['rate_per_tab']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.4f} INR per tab\n")

# =============================
# 2️⃣ Interactive Dropdowns & Prediction for Bajaj to be L1 (Product + Zone only)
# =============================

# Prepare dropdown options
product_list = encoders['product name'].classes_
zone_list = encoders['zone'].classes_

# Create dropdown widgets (only Product and Zone)
product_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(product_list)],
    description='Product:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

zone_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(zone_list)],
    description='Zone:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

output = widgets.Output()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output()

            # Selected indexes
            product_choice = product_dropdown.value
            zone_choice = zone_dropdown.value

            # Names
            product_name = product_list[product_choice]
            zone_name = zone_list[zone_choice]

            # Encode selections
            prod_code = encoders['product name'].transform([product_name])[0]
            zone_code = encoders['zone'].transform([zone_name])[0]

            # Bajaj Healthcare code (exact name match)
            bajaj_name = "Bajaj Healthcare Limited"
            if bajaj_name in encoders['company name'].classes_:
                bajaj_code = encoders['company name'].transform([bajaj_name])[0]
            else:
                print(f"Error: '{bajaj_name}' not found in company names.")
                return

            # Filter data for selected product and zone only
            comp_zone_data = df[(df['product name'] == prod_code) & (df['zone'] == zone_code)]

            if comp_zone_data.empty:
                print(f"No data available for product '{product_name}' in zone '{zone_name}'.")
                return

            # Rank companies by rate_per_tab ascending (lowest is L1)
            comp_zone_data = comp_zone_data.copy()
            comp_zone_data['rank'] = comp_zone_data.groupby(['product name', 'zone'])['rate_per_tab'].rank(method='dense', ascending=True)

            # Filter top 5 ranks only
            top_competitors = comp_zone_data[comp_zone_data['rank'] <= 5]

            # Map rank to labels
            rank_labels_map = {1: 'L1', 2: 'L2', 3: 'L3', 4: 'L4', 5: 'L5'}
            top_competitors['rank_label'] = top_competitors['rank'].astype(int).map(rank_labels_map)

            # Show ranked competitors table
            display_cols = ['company name', 'rate_per_tab', 'rank', 'rank_label']
            ranked_summary = top_competitors[display_cols].copy()
            ranked_summary['company name'] = encoders['company name'].inverse_transform(ranked_summary['company name'])
            ranked_summary = ranked_summary.sort_values('rank')

            print(f"Competitor ranks for product '{product_name}' in zone '{zone_name}':")
            display(ranked_summary.reset_index(drop=True))

            # Get lowest competitor rate_per_tab excluding Bajaj
            competitors = top_competitors[top_competitors['company name'] != bajaj_code]
            if not competitors.empty:
                l1_rate = competitors['rate_per_tab'].min()
            else:
                l1_rate = None

            # Median tabs for pack rate calculation, fallback to global median if no Bajaj data
            bajaj_data = df[(df['company name'] == bajaj_code) &
                            (df['product name'] == prod_code) &
                            (df['zone'] == zone_code)]
            tabs_val = bajaj_data['tabs'].median() if not bajaj_data.empty else df['tabs'].median()

            # Suggest Bajaj rate per tab to be L1
            if l1_rate is not None and l1_rate > 0:
                suggested_rate_per_tab = l1_rate * 0.99  # 1% lower to beat L1
            else:
                # If no competitor rates, fallback to model prediction
                qty_val = bajaj_data['qty'].median() if not bajaj_data.empty else df['qty'].median()
                region_val = int(bajaj_data['region'].median()) if not bajaj_data.empty else int(df['region'].median())
                units_val = int(bajaj_data['units'].median()) if not bajaj_data.empty else int(df['units'].median())

                input_row = {
                    'company name': bajaj_code,
                    'product name': prod_code,
                    'qty': qty_val,
                    'region': region_val,
                    'units': units_val,
                    'zone': zone_code
                }
                input_df = pd.DataFrame([input_row], columns=features)
                suggested_rate_per_tab = model.predict(input_df)[0]

            suggested_pack_rate = suggested_rate_per_tab * tabs_val

            print(f"\nCurrent L1 competitor rate per tab: {l1_rate if l1_rate is not None else 'No competitors found'}")
            print(f"Median tabs (for pack rate): {tabs_val}")
            print(f"Suggested Bajaj quote rate per tab to be L1: {suggested_rate_per_tab:.4f} INR")
            print(f"Suggested Bajaj pack rate (rate_per_tab * tabs): {suggested_pack_rate:.2f} INR")

            if not bajaj_data.empty:
                print("\nHistorical Bajaj data for this product+zone:")
                print(f" Mean rate_per_tab: {bajaj_data['rate_per_tab'].mean():.4f} INR")
                print(f" Mean pack rate: {bajaj_data['rate quoted'].mean():.2f} INR")
                print(f" Median tabs: {tabs_val}")
            else:
                print("\nNo historical Bajaj data available for this product+zone.")

# Attach observers
product_dropdown.observe(on_change)
zone_dropdown.observe(on_change)

# Display UI
display(product_dropdown, zone_dropdown, output)

# Trigger initial display
on_change({'type': 'change', 'name': 'value', 'new': product_dropdown.value})


Mean Absolute Error: 1.2390 INR per tab



Dropdown(description='Product:', layout=Layout(width='40%'), options=(('Acyclovir 200 mg', 0), ('Acyclovir 400…

Dropdown(description='Zone:', layout=Layout(width='40%'), options=(('Central', 0), ('East', 1), ('North', 2), …

Output()

In [ ]:
# =============================
# 1️⃣ Imports
# =============================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import ipywidgets as widgets
from IPython.display import display, clear_output

# =============================
# 2️⃣ Load dataset (update path if needed)
# =============================
CSV_PATH = "/content/SampleData1.csv"   # <-- change if your file has a different path/name
df_raw = pd.read_csv(CSV_PATH)

# =============================
# 3️⃣ Clean & preprocess
# =============================
df = df_raw.copy()
df.columns = df.columns.str.strip().str.lower()

# Ensure necessary columns exist
required_cols = ['company name','product name','rate quoted','tabs','qty','region','units','zone']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Clean numeric
df['tabs'] = pd.to_numeric(df['tabs'], errors='coerce')
df['qty']  = pd.to_numeric(df['qty'], errors='coerce')

# Drop unusable rows
df = df.dropna(subset=['rate quoted','tabs'])
df = df[df['tabs'] > 0]

# Create per-tab target
df['rate_per_tab'] = df['rate quoted'] / df['tabs']
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['rate_per_tab'])

# Remove extreme outliers on target (robust)
low_q, high_q = df['rate_per_tab'].quantile([0.01, 0.99])
df = df[(df['rate_per_tab'] >= low_q) & (df['rate_per_tab'] <= high_q)]

# =============================
# 4️⃣ Encode categoricals
# =============================
cat_cols = ['company name','product name','region','units','zone']
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le

# Keep a copy of human-readable names for later display
company_names = encoders['company name'].classes_
product_names = encoders['product name'].classes_
zone_names    = encoders['zone'].classes_

# =============================
# 5️⃣ Train model (predict rate_per_tab)
# =============================
# Features (exclude 'tabs' so per-tab is not driven by pack size)
features = ['company name','product name','qty','region','units','zone']
X = df[features]
y = df['rate_per_tab']

# Split & train
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = RandomForestRegressor(n_estimators=300, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Model MAE (rate per tab): {mae:.4f} INR/tab\n")

# =============================
# 6️⃣ Helper functions
# =============================
def median_or_global(series_hist, series_global):
    """Median from history if available, else global median."""
    if series_hist.size > 0 and not np.isnan(series_hist.median()):
        return float(series_hist.median())
    return float(series_global.median())

def competitor_rank_table(df_subset):
    """
    Build a competitor ranking table for a given (product, zone) subset.
    Uses median rate_per_tab per company to avoid multiple row noise.
    """
    # Median per company within this product+zone
    agg = (df_subset
           .groupby('company name', as_index=False)['rate_per_tab']
           .median()
           .rename(columns={'rate_per_tab':'median_rate_per_tab'}))

    # Rank ascending => L1 is smallest median rate
    agg['rank'] = agg['median_rate_per_tab'].rank(method='dense', ascending=True).astype(int)

    # Label L1-L5 for ranks 1..5, else keep rank number
    rank_labels_map = {1:'L1', 2:'L2', 3:'L3', 4:'L4', 5:'L5'}
    agg['rank_label'] = agg['rank'].map(rank_labels_map).fillna(agg['rank'].astype(str))

    # Add names back
    agg['company name'] = encoders['company name'].inverse_transform(agg['company name'])

    # Sort by rank then company name for consistent display
    agg = agg.sort_values(['rank','median_rate_per_tab','company name']).reset_index(drop=True)
    return agg

def suggest_bhl_rate_per_tab(df_subset, beat_pct=0.01):
    """
    Suggest a competitive per-tab rate slightly below the L1 competitor
    for the given (product+zone) subset, excluding Bajaj.
    Uses a robust baseline (max of 25th percentile and min) then applies beat_pct.
    """
    # Identify Bajaj code (if present)
    bhl_exact = "Bajaj Healthcare Limited"
    if bhl_exact in company_names:
        bhl_code = encoders['company name'].transform([bhl_exact])[0]
    else:
        bhl_code = None

    # Exclude BHL from competitor pool if present
    if bhl_code is not None:
        comp = df_subset[df_subset['company name'] != bhl_code].copy()
    else:
        comp = df_subset.copy()

    if comp.empty:
        return None  # no competitor info

    # Robust competitive baseline
    q25 = comp['rate_per_tab'].quantile(0.25)
    minv = comp['rate_per_tab'].min()
    baseline = max(q25, minv)

    # Beat by given percentage
    suggested = baseline * (1 - float(beat_pct))
    return float(suggested)

def clip_to_history(value, hist_series, low=0.05, high=0.95):
    """Clip value to the given percentiles of historical series, if available."""
    if hist_series.size < 3:
        return value
    lo = hist_series.quantile(low)
    hi = hist_series.quantile(high)
    return float(np.clip(value, lo, hi))

# =============================
# 7️⃣ Widgets (UI)
# =============================
product_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(product_names)],
    description='Product:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

zone_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(zone_names)],
    description='Zone:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

company_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(company_names)],
    description='Company:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

mode_toggle = widgets.ToggleButtons(
    options=[('Model Prediction', 'model'),
             ('BHL Slightly Below L1', 'bhl')],
    description='Mode:',
    style={'description_width': 'initial'}
)

beat_slider = widgets.FloatSlider(
    value=0.01,  # 1%
    min=0.0, max=0.10, step=0.0025,
    description='Beat L1 by (fraction):',
    readout_format='.3f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

# =============================
# 8️⃣ Interaction logic
# =============================
def on_change(_change):
    with output:
        clear_output()

        # Selected items
        prod_idx = product_dropdown.value
        zone_idx = zone_dropdown.value
        comp_idx = company_dropdown.value
        mode     = mode_toggle.value
        beat_pct = beat_slider.value

        product_name = product_names[prod_idx]
        zone_name    = zone_names[zone_idx]
        company_name = company_names[comp_idx]

        prod_code = encoders['product name'].transform([product_name])[0]
        zone_code = encoders['zone'].transform([zone_name])[0]
        comp_code = encoders['company name'].transform([company_name])[0]

        # Subsets
        prod_zone = df[(df['product name']==prod_code) & (df['zone']==zone_code)]
        if prod_zone.empty:
            print(f"No data for product '{product_name}' in zone '{zone_name}'.")
            return

        # Show competitor ranks (median per company)
        print(f"Competitor ranks — Product: {product_name} | Zone: {zone_name}")
        ranked = competitor_rank_table(prod_zone)
        display(ranked)

        # Tabs for pack calculation (prefer company+product+zone, then product+zone, then global)
        hist_cpz = df[(df['company name']==comp_code) &
                      (df['product name']==prod_code) &
                      (df['zone']==zone_code)]
        if hist_cpz.empty:
            tabs_val = median_or_global(prod_zone['tabs'], df['tabs'])
        else:
            tabs_val = median_or_global(hist_cpz['tabs'], prod_zone['tabs'])

        # Prepare a baseline row to predict (for "model" mode)
        # Use medians from historical when available, fall back to product+zone then global
        def med_or(df1, col):
            return median_or_global(df1[col], prod_zone[col] if col in prod_zone.columns else df[col])

        qty_val   = med_or(hist_cpz, 'qty')
        region_val= med_or(hist_cpz, 'region')
        units_val = med_or(hist_cpz, 'units')

        # BHL code (if needed)
        bhl_exact = "Bajaj Healthcare Limited"
        bhl_present = bhl_exact in company_names
        bhl_code = encoders['company name'].transform([bhl_exact])[0] if bhl_present else None

        # Decide company for prediction depending on mode
        if mode == 'bhl':
            if not bhl_present:
                print(f"Note: '{bhl_exact}' not found in data. Falling back to model mode for selected company.")
                mode_use = 'model'
                company_for_pred = comp_code
            else:
                mode_use = 'bhl'
                company_for_pred = bhl_code
        else:
            mode_use = 'model'
            company_for_pred = comp_code

        # Compute suggestion
        if mode_use == 'bhl':
            # Suggest slightly below L1 (robust baseline)
            suggested_rpt = suggest_bhl_rate_per_tab(prod_zone, beat_pct=beat_pct)
            if suggested_rpt is None:
                # No competitors -> fall back to model prediction with BHL as company
                input_row = {
                    'company name': company_for_pred,
                    'product name': prod_code,
                    'qty': qty_val,
                    'region': region_val,
                    'units': units_val,
                    'zone': zone_code
                }
                input_df = pd.DataFrame([input_row], columns=features)
                suggested_rpt = float(model.predict(input_df)[0])

            # Clip to BHL historical if exists, else clip to product+zone distribution
            bhl_hist = df[(df['company name']==company_for_pred) &
                          (df['product name']==prod_code) &
                          (df['zone']==zone_code)]
            if not bhl_hist.empty:
                suggested_rpt = clip_to_history(suggested_rpt, bhl_hist['rate_per_tab'])
            else:
                suggested_rpt = clip_to_history(suggested_rpt, prod_zone['rate_per_tab'])

            pack_rate = suggested_rpt * tabs_val

            print("\nMode: BHL Slightly Below L1")
            print(f"Beat L1 by: {beat_pct*100:.2f}%")
            # Current L1 (median-based) among competitors (exclude BHL)
            comp_ex_bhl = prod_zone[prod_zone['company name'] != bhl_code] if bhl_present else prod_zone
            if not comp_ex_bhl.empty:
                l1_now = comp_ex_bhl.groupby('company name')['rate_per_tab'].median().min()
                print(f"Current L1 (median) per-tab among competitors: {l1_now:.4f} INR/tab")
            else:
                print("No competitor rates found; used model fallback.")

            print(f"Tabs used for pack calc: {tabs_val}")
            print(f"Suggested BHL per-tab rate: {suggested_rpt:.4f} INR/tab")
            print(f"Suggested BHL pack rate: {pack_rate:.2f} INR")

        else:
            # Standard model prediction for selected company
            input_row = {
                'company name': company_for_pred,
                'product name': prod_code,
                'qty': qty_val,
                'region': region_val,
                'units': units_val,
                'zone': zone_code
            }
            input_df = pd.DataFrame([input_row], columns=features)
            pred_rpt = float(model.predict(input_df)[0])

            # Clip to company+product+zone history if exists; else to product+zone distribution
            if not hist_cpz.empty:
                pred_rpt = clip_to_history(pred_rpt, hist_cpz['rate_per_tab'])
            else:
                pred_rpt = clip_to_history(pred_rpt, prod_zone['rate_per_tab'])

            pack_rate = pred_rpt * tabs_val

            print("\nMode: Model Prediction")
            print(f"Company: {company_name}")
            print(f"Tabs used for pack calc: {tabs_val}")
            print(f"Predicted per-tab rate: {pred_rpt:.4f} INR/tab")
            print(f"Predicted pack rate: {pack_rate:.2f} INR")

# Hook up observers
for w in (product_dropdown, zone_dropdown, company_dropdown, mode_toggle, beat_slider):
    w.observe(on_change, names='value')

# Show UI
display(
    widgets.HBox([product_dropdown, zone_dropdown]),
    widgets.HBox([company_dropdown]),
    widgets.HBox([mode_toggle, beat_slider]),
    output
)

# Trigger initial render
on_change(None)


Model MAE (rate per tab): 1.2146 INR/tab



Output()